In [ ]:
# Quadratic Equation Solver Workflow
# This notebook demonstrates conditional branching in LangGraph to solve quadratic equations
# The workflow determines the type of roots and computes them accordingly

# Import required modules
from langgraph.graph import StateGraph, START, END  # LangGraph workflow components
from typing import TypedDict, Literal               # Type-safe state definition

In [ ]:
# Define the Quadratic Equation State Schema
# Tracks coefficients, equation string, discriminant, and result
class QuadState(TypedDict):
    a : int                # Coefficient of x^2
    b : int                # Coefficient of x
    c : int                # Constant term
    equation : str         # String representation of the equation
    discriminant : float   # Discriminant value
    result : str           # Result (roots or no real roots)

In [ ]:
def show_equation(state: QuadState) -> QuadState:
    """
    Step 1: Build the quadratic equation string from coefficients
    """
    equation = f"{state['a']}x^2{state['b']}x{state['c']}"
    state['equation'] = equation
    return state

def calculate_discriminant(state: QuadState) -> QuadState:
    """
    Step 2: Calculate the discriminant (b^2 - 4ac)
    """
    discriminant = state['b']**2 - (4*state['a']*state['c'])
    state['discriminant'] = discriminant
    return state

def no_real_roots(state: QuadState) -> QuadState:
    """
    Step 3a: Handle case with no real roots
    """
    state['result'] = 'no real roots'
    return state

def real_roots(state: QuadState) -> QuadState:
    """
    Step 3b: Compute and display two distinct real roots
    """
    root1 = (-state['b'] + state['discriminant']**0.5)/(2*state['a'])
    root2 = (-state['b'] - state['discriminant']**0.5)/(2*state['a'])
    result = f'The roots are {root1} and {root2}'
    state['result'] = result
    return state

def repeated_roots(state: QuadState) -> QuadState:
    """
    Step 3c: Compute and display the repeated root
    """
    root = -state['b']/(2*state['a'])
    result = f'Only repeating root is : {root}'
    state['result'] = result
    return state

def check_condition(state: QuadState) -> Literal['real_roots', 'no_real_roots', 'repeated_roots'] :
    """
    Step 4: Conditional branching based on discriminant value
    """
    if state['discriminant'] > 0:
        return 'real_roots'
    elif state['discriminant'] == 0:
        return 'repeated_roots'
    else:
        return 'no_real_roots'

In [ ]:
# Build the Conditional Quadratic Solver Workflow Graph
# The workflow branches based on the discriminant to handle all root cases

graph = StateGraph(QuadState)

# Add nodes for each step
graph.add_node('show_equation', show_equation)              # Build equation string
graph.add_node('calculate_discriminant', calculate_discriminant)  # Calculate discriminant
graph.add_node('no_real_roots', no_real_roots)              # Handle no real roots
graph.add_node('real_roots', real_roots)                    # Handle two real roots
graph.add_node('repeated_roots', repeated_roots)            # Handle repeated root

# Define edges for sequential and conditional flow
graph.add_edge(START, 'show_equation')
graph.add_edge('show_equation', 'calculate_discriminant')
graph.add_conditional_edges('calculate_discriminant', check_condition)
graph.add_edge('real_roots', END)
graph.add_edge('repeated_roots', END)
graph.add_edge('no_real_roots', END)

# Compile the graph into an executable workflow
workflow = graph.compile()

In [ ]:
# Execute the Workflow with Sample Coefficients
# Test the quadratic solver for a sample equation
initial_state = {
    'a' : 4,    # Coefficient of x^2
    'b' : -5,   # Coefficient of x
    'c' : -4    # Constant term
}

# Run the workflow and display results
workflow.invoke(initial_state)

{'a': 4,
 'b': -5,
 'c': -4,
 'equation': '4x^2-5x-4',
 'discriminant': 89,
 'result': 'The roots are 6.179247641507075 and 3.820752358492925'}

In [ ]:
# Visualize the Workflow Structure
# Shows conditional branching for root cases
print(workflow.get_graph().draw_ascii())

                             +-----------+                               
                             | __start__ |                               
                             +-----------+                               
                                   *                                     
                                   *                                     
                                   *                                     
                           +---------------+                             
                           | show_equation |                             
                           +---------------+                             
                                   *                                     
                                   *                                     
                                   *                                     
                      +------------------------+                         
                      | calculate_disc